In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time

import warnings
warnings.filterwarnings('ignore')

import string, nltk, re
from konlpy.tag import Okt
from wordcloud import WordCloud

from PIL import Image
from wordcloud import ImageColorGenerator

import os
driver_name =  os.environ['HOMEDRIVE'] + os.environ['HOMEPATH'] + '\\Downloads\\chromedriver_win32\\chromedriver.exe'

### 영화관입장권통합전산망 KOBIS 에서 영화 정보 크롤링

In [2]:
url = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

home_url = "https://www.kobis.or.kr/"

In [3]:
driver = webdriver.Chrome(driver_name)
driver.get(url)
time.sleep(2)

In [4]:
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
print(len(lis))

10


In [6]:
driver = webdriver.Chrome(driver_name)
driver.get(url)
time.sleep(2)

TOTAL_COUNT = 1000
START_COUNT = 0
SUB_COUNT = 0
for i in range(START_COUNT):
    driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
    time.sleep(3)
    
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
current = 0
for k in range(SUB_COUNT):
    current += 1
    lis[current].click()
    time.sleep(3)
    lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

In [5]:
TOTAL_COUNT = 1000

In [6]:

# driver = webdriver.Chrome(driver_name)

# driver.get(url)
# time.sleep(2)
ORDER = 'movie카'
n_page = 0
# paging
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
# print(lis)

end, current = len(lis), 0
lines = []

while True:

    try:
        trs = driver.find_elements(By.CSS_SELECTOR, 'tbody > tr')
    except:
        print('nPgae = ', n_page)
        break

    for tr in trs:

        td = tr.find_elements(By.TAG_NAME, 'td')

        kind = td[5].text
        genre = td[6].text
        
        if kind =='' or kind =='단편' or kind == "기타" or kind == '옴니버스':
            continue
        if '성인' in genre or genre == '' :
            continue

        td[0].find_element(By.TAG_NAME, 'a').click()
        time.sleep(3)

        # 영화 제목
        title = driver.find_element(By.CSS_SELECTOR, '.hd_layer > div').text
        find_index = title.find(')') 
        if find_index >= 0 :
            title = title[:find_index+1]
        
        infos = driver.find_element(By.CSS_SELECTOR, 'div.basic')

        # 시놉시스가 없으면 pass
        try :
            synopsis = infos.find_element(By.CSS_SELECTOR, '.info.info2 > p').text
        except :
            driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
            time.sleep(3)
            continue

        infos_head = infos.find_element(By.CSS_SELECTOR, '.ovf.info.info1')
            
        # 이미지 URL
        img = infos_head.find_element(By.CSS_SELECTOR, 'a > img').get_attribute('src')

        # 요약 정보
        dds = infos_head.find_elements(By.CSS_SELECTOR, 'dl > dd')

        # 코드 정보
        code = dds[0].text
        summering = dds[3].text
        if (summering.find('멜로/로맨스') >=0 or summering.find('드라마') >=0 ) and summering.find('청소년관람불가') >= 0:
            driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
            time.sleep(3)
            continue

        # 개봉일
        first_day = dds[-6].text
        # 제작년도
        production_year = dds[-5].text
            
        # 감독 / 주연 배우
        tmp = infos.find_elements(By.CSS_SELECTOR, '.info.info2 > .staffMore > dl > div')


        movie_director, star_actor = '', ''
        find_count = 0

        # 감독 : 있는 경우 : 첫 번째에 감독이 없으면 감독이 없는 데이터다
        if len(tmp) > 0 :
            nindex = tmp[0].text.find('감독')
            if nindex >= 0:
                movie_director = tmp[0].text[nindex + 2:].strip()
                find_count += 1

        # 주연 배우
        for tp in tmp[find_count:]:
            
            text = tp.text.split('\n')            
            if len(text) < 2 : continue

            nfind = text[1].find('[주연]')
            
            if nfind >= 0:
                star_actor = text[1][nfind + 4: ].strip()
                # print(star_actor)
                break

        # print(title, ' - ', movie_director, ' - ', star_actor)
        lines.append({'code': code, 'title': title, 'genre': genre, 'summering': summering, 
                    'first_day': first_day, 'production_year': production_year, 
                    'movie_director':movie_director, 'star_actor': star_actor, 
                    'img': img, 'synopsis': synopsis})

        driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
        time.sleep(3)


    current += 1
    n_page += 1

    # if n_page == TOTAL_COUNT: break

    if end == current:
        try:
            driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
            time.sleep(3)
            lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
            end, current = len(lis), 0
        except:
            print('except2 nPgae = ', n_page)
            break
    else:
        # 다음 페이지 넘어가기
        lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
        # print(lis)
        lis[current].click()
        time.sleep(3)

df = pd.DataFrame(lines)

df.to_csv(f'../static/data/{ORDER}.csv', index=False)

except2 nPgae =  72


In [5]:
len(lines)

332

In [68]:
df

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20226897,그녀의 취미생활 (Her Hobby),미스터리(2),"장편 | 일반영화 | 미스터리, 범죄, 드라마 | 한국",2023,2022년,하명미,정이서 | 김혜나,https://www.kobis.or.kr/kobis/web/comm/images/...,남편으로부터 가정폭력을 겪은 정인은 그토록 떠나고 싶었던 고향 ‘박하마을’로 돌아오...
1,20228254,그녀가 말했다 (She Said),드라마,장편 | 일반영화 | 드라마 | 129분 0초 | 15세이상관람가 | 미국,2022-11-30,2022년,마리아 슈라더,캐리 멀리건 | 조 카잔 | 패트리샤 클락슨 | 안드레 브라우퍼,https://www.kobis.or.kr/common/mast/movie/2022...,“이것은 실화다!”\n\n미국의 대표 일간지 뉴욕타임스의 탐사 보도 취재 기자 메건...
2,20188462,감쪽같은 그녀 (A Little Princess),드라마,장편 | 일반영화 | 드라마 | 104분 25초 | 전체관람가 | 한국,2019-12-04,2019년,허인무,나문희(말순) | 김수안(공주),https://www.kobis.or.kr/common/mast/movie/2019...,“초면에 실례하겠습니다~”\n72살 나 홀로 라이프를 즐기는 ‘말순’ 할매의 인생에...
3,20205378,그녀의 비밀정원 (Invitation),드라마(1),"장편 | 예술영화 | 드라마, 멜로/로맨스 | 87분 33초 | 15세이상관람가 | 한국",2020-04-22,2019년,김인식,예지원(장현재) | 최우제(한장서) | 김동길(한동주),https://www.kobis.or.kr/common/mast/movie/2020...,“진실을 알고 싶을 뿐입니다”\n\n형 ‘한장서’와 동생 ‘한충서’.\n두 형제는 ...
4,20202982,"가끔 그녀를 생각한다 (Sometimes, I Remind Her)",드라마,장편 | 일반영화 | 드라마 | 16분 0초,해당정보없음,1997년,박경목,,https://www.kobis.or.kr/kobis/web/comm/images/...,가정주부의 소외를 통해 보이지 않는 가 족 간의 균열을 포착해 내고 있다.
5,20081616,그녀들의 방 (The Room Nearby),드라마,장편 | 예술영화 | 드라마 | 106분 0초 | 15세이상관람가 | 한국,2009-03-12,2008년,고태정,정유미(구언주) | 예수정(유석희) | 김민정 | 김지나(이수진 ),https://www.kobis.or.kr/upload/up_img/cleansin...,학습지 교사 언주는 가난이 지긋지긋하다. 고시원에 거주하면서 악착같이 돈을 모으는 ...
6,20208474,그녀의 비밀정원-무삭제판,멜로/로맨스(1),"장편 | 일반영화 | 멜로/로맨스, 드라마 | 88분 17초 | 한국",해당정보없음,2020년,김인식,예지원 | 최우제,https://www.kobis.or.kr/common/mast/movie/2020...,“진실을 알고 싶을 뿐입니다”\n\n형 ‘한장서’와 동생 ‘한충서’.\n두 형제는 ...
7,20198932,그녀가 사랑했던 이야기 (The Quilt),드라마,"장편 | 일반영화 | 드라마 | 84분 0초 | 프랑스, 인도",해당정보없음,2018년,,,https://www.kobis.or.kr/kobis/web/comm/images/...,"작가 이스마트 추그타이의 동명 소설, 『리하프』를 원작으로 한 작품. 이스마트 추그..."
8,20148748,그녀 (Her),드라마(1),"장편 | 예술영화 | 드라마, 멜로/로맨스 | 125분 57초 | 15세이상관람가 ...",2013년,개봉,스파이크 존즈,호아킨 피닉스 | 스칼렛 요한슨 | 에이미 아담스 | 루니 마라,https://www.kobis.or.kr/common/mast/movie/2019...,"사랑과 관계에 서툰 모두를 위한 감성 로맨스\n우리에게 사랑하는 법을 알려준, 그녀..."
9,20122782,그녀가 부른다 (Do You Hear She Sings?),멜로/로맨스(1),"장편 | 예술,독립 영화 | 멜로/로맨스, 드라마 | 96분 43초 | 15세이상관...",2013-12-26,2012년,박은형,윤진서(진경) | 오민석(경호),https://www.kobis.or.kr/common/mast/movie/2013...,“모든 사람이 다 똑같이 살 필요는 없잖아요.”\n남들 시선에 맞추어 사는 인생은 ...


In [67]:
df.shape

(10, 10)

In [10]:
df = pd.DataFrame(lines)

In [7]:
driver.close()

In [62]:
df.tail()

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,2023C022,나의 엽기적인 그녀 (My Sassy Girl),멜로/로맨스(2),"온라인전용 | 일반영화 | 멜로/로맨스, 코미디, 액션 | 94분 44초 | 중국",해당정보없음,2021년,,,https://www.kobis.or.kr/common/mast/movie/2023...,국숫집을 운영하는 성실한 청년 뉴둬둬. 여느 날처럼 배달을 마치고 가게로 돌아오는데...
1,20200934,"나, 그, 그녀 (Me Him Her)",코미디(1),"장편 | 일반영화 | 코미디, 멜로/로맨스 | 97분 51초 | 미국",해당정보없음,2015년,,더스틴 밀리건 | 루크 브레이시,https://www.kobis.or.kr/common/mast/movie/2020...,"직업 없이 떠돌이 생활을 하는 한량 ‘코리’는\n가장 친한 친구이자, TV쇼 라이징..."
2,20199641,남자에 굶주린 그녀,드라마,장편 | 일반영화 | 드라마 | 55분 39초 | 청소년관람불가 | 한국,2020-01-10,2019년,박은석,,https://www.kobis.or.kr/kobis/web/comm/images/...,첫 번째 이야기. 중국어 공부를 하기 위해 과외를 하게 된 병섭. 하지만 과외 선생...
3,20191744,노리개: 그녀의 눈물 (Norigae),드라마,장편 | 일반영화 | 드라마 | 101분 55초 | 15세이상관람가 | 한국,2019-06-13,2019년,최승호,마동석(이장호) | 이승연(김미현) | 민지현(정지희) | 변요한(박지훈),https://www.kobis.or.kr/common/mast/movie/2019...,"2009년 충격적인 사건, 그리고 10년\n끝나지 않은 충격 실화\n\n여배우의 죽..."
4,20121864,나인뮤지스; 그녀들의 서바이벌 (9 Muses of Star Empire),다큐멘터리,장편 | 예술영화 | 다큐멘터리 | 82분 7초 | 12세이상관람가 | 한국,2014-09-25,2011년,이학준,나인뮤지스 | 현주(9 Muses members),https://www.kobis.or.kr/common/mast/movie/2014...,"""나인뮤지스""는 질투와 배신, 그리고 스캔들로 가득한 비정한 쇼비즈니스계에서 부와 ..."


In [12]:
df.to_csv(f'../static/data/{ORDER}.csv', index=False)